# NLMD
https://scikit-image.org/docs/stable/auto_examples/filters/plot_nonlocal_means.html#sphx-glr-auto-examples-filters-plot-nonlocal-means-py

## Process NLMD Multiple File

In [1]:
import os
import numpy as np
import pydicom

import numpy as np
import matplotlib.pyplot as plt
from skimage import img_as_float, exposure
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise
import os


import pydicom
import skimage
print(skimage.__version__)

import cv2

0.21.0


In [ ]:
import time
import psutil
import tracemalloc
import pynvml
import tensorflow as tf
import threading

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

cpu_utilization_samples = []
stop_sampling = False

def sample_cpu_utilization(interval=1.0):
    while not stop_sampling:
        cpu_utilization_samples.append(psutil.cpu_percent(interval=interval))

sampling_thread = threading.Thread(target=sample_cpu_utilization)
sampling_thread.start()

start_time = time.time()
tracemalloc.start()
process = psutil.Process()
cpu_times_start = process.cpu_times()

In [ ]:
def denoise_image(image):
    """
    Apply Non-Local Means Denoising to a given image.
    """
    image_float = img_as_float(image)
    
    
    patch_kw = dict(
        patch_size = 5,
        patch_distance= 7 
    )
    
    denoised_image = denoise_nl_means(image_float, fast_mode=False, **patch_kw)
    
    return denoised_image

def process_dicom_images(input_dir, output_dir):
    """
    Process all DICOM files in the input directory and its subdirectories,
    apply Non-Local Means Denoising, and save the denoised images to the output directory.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, _, files in os.walk(input_dir):
        for filename in files:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                try:
                    dicom = pydicom.dcmread(file_path)
                    image = dicom.pixel_array
                    
                    min_val = image.min()
                    max_val = image.max()
                    
                    denoised_image = denoise_image(image)
                    
                    denoised_rescaled = (denoised_image - denoised_image.min()) / \
                                        (denoised_image.max() - denoised_image.min())
                    denoised_rescaled = denoised_rescaled * (max_val - min_val) + min_val
                    
                    dicom.PixelData = denoised_rescaled.astype('uint16').tobytes()
                    
                    relative_path = os.path.relpath(root, input_dir)
                    sub_output_dir = os.path.join(output_dir, relative_path)
                    if not os.path.exists(sub_output_dir):
                        os.makedirs(sub_output_dir)
                    output_path = os.path.join(sub_output_dir, filename)
                    
                    dicom.save_as(output_path)
                    print(f"Processed: {file_path} -> {output_path}")
                except Exception as e:
                    print(f"Skipping {file_path}: Not a valid DICOM file ({e})")

input_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB"
output_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original"
process_dicom_images(input_directory, output_directory)

Processed: C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB\FCAVDP00.dcm -> C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original\.\FCAVDP00.dcm
Processed: C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB\FCAVDP0A.dcm -> C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original\.\FCAVDP0A.dcm
Processed: C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB\FCAVDP0C.dcm -> C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original\.\FCAVDP0C.dcm
Processed: C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Data 10/Original/CATHLAB\FCAVDP0E.dcm -> C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (4 apr)/Results 10/NLMD57/Original\.\FCAVDP0E.dcm
Processed: C:/Matthe

In [ ]:
cpu_times_end = process.cpu_times()
end_time = time.time()
stop_sampling = True
sampling_thread.join()

current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
utilization = pynvml.nvmlDeviceGetUtilizationRates(handle)
pynvml.nvmlShutdown()

gpu_mem_used = mem_info.used / 1024**2 
gpu_util_percent = utilization.gpu    

execution_time = end_time - start_time
cpu_time_used = (cpu_times_end.user + cpu_times_end.system) - (cpu_times_start.user + cpu_times_start.system)
cpu_usage_percent = (cpu_time_used / execution_time) * 100 if execution_time > 0 else 0

avg_cpu_utilization = sum(cpu_utilization_samples) / len(cpu_utilization_samples) if cpu_utilization_samples else 0

ram_usage = peak / 10**6 

print(f"Execution Time: {execution_time:.2f} seconds")
print(f"CPU Usage (Script): {cpu_usage_percent:.2f}%")
print(f"CPU Utilization (System Avg): {avg_cpu_utilization:.2f}%")
print(f"Peak RAM Usage: {ram_usage:.2f} MB")
print(f"GPU Memory Used: {gpu_mem_used:.2f} MB")
print(f"GPU Utilization: {gpu_util_percent:.2f}%")
pynvml.nvmlShutdown()

Execution Time: 22.30 seconds
CPU Usage (Script): 97.90%
CPU Utilization (System Avg): 7.26%
Peak RAM Usage: 3.86 MB
GPU Memory Used: 775.68 MB
GPU Utilization: 2.00%
